In [1]:
#I got this code from 'project detail' in Data Wrangling course.
#And I revised a little to fit for Python3 and my computer.
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import xml.etree.cElementTree as ET  # Use cElementTree or lxml if too slow

OSM_FILE = "C:\\projects\\New_York_OSM\\New_York.osm"  # Replace this with your osm file
SAMPLE_FILE = "C:\\projects\\New_York_OSM\\New_York_sample.osm"

k = 30 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    print(root.tag, root.attrib)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n'.encode('utf-8'))
    output.write('<osm>\n  '.encode('utf-8'))

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>'.encode('utf-8'))

osm {'version': '0.6', 'generator': 'osmconvert 0.8.5', 'timestamp': '2017-09-03T15:00:02Z'}


In [7]:
#I got this code from Data Wrangling course.
import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET

#import cerberus

#import schema

OSM_PATH = "C:\\projects\\New_York_OSM\\New_York_sample.osm"

NODES_PATH = "C:\\projects\\New_York_OSM\\sample_nodes.csv"
NODE_TAGS_PATH = "C:\\projects\\New_York_OSM\\sample_nodes_tags.csv"
WAYS_PATH = "C:\\projects\\New_York_OSM\\sample_ways.csv"
WAY_NODES_PATH = "C:\\projects\\New_York_OSM\\sample_ways_nodes.csv"
WAY_TAGS_PATH = "C:\\projects\\New_York_OSM\\sample_ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

#SCHEMA = schema.schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    # YOUR CODE HERE
    if element.tag == 'node':
        for key, value in element.attrib.items():
            if key in node_attr_fields:
                if key == 'id':
                    nodeId = value
                node_attribs[key] = value
            #print('node')
        for sub in element:
            key, value = sub.attrib['k'], sub.attrib['v']
            #print 'node tag', key, value
            if not problem_chars.search(key):
                
                if LOWER_COLON.match(key):
                    left, right = key.split(":", 1)
                    tags.append({"id" : nodeId, "key" : right, "value" : value, "type" : left})
                    #print('node tag', left, right, value)
                else:
                    tags.append({"id" : nodeId, "key" : key, "value" : value, "type" : default_tag_type})
                    #print('node tag', default_tag_type, key, value)
        #print node_attribs, tags        
        return {'node': node_attribs, 'node_tags': tags}
    
    elif element.tag == 'way':
        for key, value in element.attrib.items():
            if key in WAY_FIELDS:
                if key == 'id':
                    wayId = value
                way_attribs[key] = value
                #print('way')
        pos = 0
        for sub in element:
            if sub.tag == 'tag':
                key, value = sub.attrib['k'], sub.attrib['v']
                if not problem_chars.search(key):
                    
                    if LOWER_COLON.match(key):
                        left, right = key.split(":", 1)
                        tags.append({"id" : wayId, "key" : right, "value" : value, "type" : left})
                        #print('way tag', left, right, value)
                    else:
                        tags.append({"id" : wayId, "key" : key, "value" : value, "type" : default_tag_type})
                        #print('way tag', default_tag_type, key, value)
            else:
                ref = sub.attrib['ref']
                way_nodes.append({"id" : wayId, "node_id" : ref, "position" : pos})
                #print('way nodes', ref, pos)
                pos += 1
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}


# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=None):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))

'''
class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.items()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)
'''

# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with open(NODES_PATH, 'w', encoding='UTF-8', newline='') as nodes_file, \
         open(NODE_TAGS_PATH, 'w', encoding='UTF-8', newline='') as nodes_tags_file, \
         open(WAYS_PATH, 'w', encoding='UTF-8', newline='') as ways_file, \
         open(WAY_NODES_PATH, 'w', encoding='UTF-8', newline='') as way_nodes_file, \
         open(WAY_TAGS_PATH, 'w', encoding='UTF-8', newline='') as way_tags_file:

        nodes_writer = csv.DictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = csv.DictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = csv.DictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = csv.DictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = csv.DictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        #validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                #if validate is True:
                #    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSM_PATH, validate=False)

- I got some help from [this example](https://gist.github.com/carlward/54ec1c91b62a5f911c42#file-sample_project-md).

In [2]:
#Connect to sampleosm.db using sqlite3.
#First, query what are the keys of tags.
import sqlite3

db = sqlite3.connect("C:\\Users\\JehyeonHeo\\sqlite_windows\\sampleosm.db")
c = db.cursor()
query = "select distinct tags.key as key \
        from (select * from nodes_tags \
                union all \
                select * from ways_tags) tags \
        order by key;"
c.execute(query)
rows = c.fetchall()
for row in rows:
    print(row)

('Bergen_County_database_ref',)
('FIXME',)
('Fax',)
('HFCS',)
('NHD:AUXCODE',)
('NHD:BldgAddr_1',)
('NHD:BldgName_1',)
('NHD:BldgNum',)
('NHD:CampusCo_1',)
('NHD:City_1',)
('NHD:ComID',)
('NHD:FCode',)
('NHD:FDate',)
('NHD:FTYPE',)
('NHD:FType',)
('NHD:Lat_1',)
('NHD:Long_1',)
('NHD:RESOLUTION',)
('NHD:ReachCode',)
('NHD:State_1',)
('NHD:Zip_1',)
('NHD:way_id',)
('NHS',)
('NJDOT_SRI',)
('NOTE',)
('NYDEC_Lands:CATEGORY',)
('NYDEC_Lands:COUNTY',)
('NYDEC_Lands:FACILITY',)
('abandoned',)
('access',)
('admin_level',)
('aerialway',)
('aeroway',)
('agricultural',)
('alt_name',)
('amenity',)
('area',)
('artist_name',)
('artwork_type',)
('atc',)
('atm',)
('attraction',)
('attribution',)
('auxcode',)
('backrest',)
('backward',)
('bar',)
('barrier',)
('basin',)
('beauty',)
('bell',)
('bench',)
('bicycle',)
('bicycle_parking',)
('bidirectional',)
('bin',)
('bitcoin',)
('black_owned',)
('board_type',)
('boat',)
('border_type',)
('both_ways',)
('boundary',)
('boundary_type',)
('brand',)
('bridge',)

In [3]:
#query what are the min and max value of latitude and longitude.
query = "select min(lat), max(lat), min(lon), max(lon) \
        from nodes;"
c.execute(query)
rows = c.fetchall()
for row in rows:
    print(row)

(40.345, 41.0969983, -74.5009765, -73.2260147)


- It looked like the latitudes and longitudes are in right range compared to this [wiki page](https://en.wikipedia.org/wiki/New_York_(state)).

In [4]:
#query about postcodes.
query = "select tags.value, count(*) as count \
        from (select * from nodes_tags \
                union all \
                select * from ways_tags) tags \
        where tags.key = 'postcode' \
        group by tags.value \
        order by count desc;"
c.execute(query)
rows = c.fetchall()
for row in rows:
    print(row)

('10314', 769)
('11234', 678)
('10312', 594)
('10306', 534)
('11236', 510)
('11706', 507)
('11385', 502)
('11746', 477)
('11743', 468)
('11757', 438)
('11229', 411)
('11203', 398)
('10469', 388)
('11207', 369)
('11434', 366)
('10305', 362)
('11208', 360)
('11223', 360)
('11704', 355)
('11357', 346)
('11204', 345)
('11731', 342)
('10466', 328)
('11221', 325)
('11214', 324)
('11215', 323)
('10465', 319)
('11413', 319)
('11210', 315)
('11725', 315)
('11219', 314)
('10309', 311)
('11358', 307)
('11220', 306)
('11420', 306)
('10304', 302)
('11209', 300)
('11412', 300)
('11377', 298)
('11379', 296)
('11717', 292)
('10308', 290)
('11235', 284)
('10301', 283)
('10461', 283)
('11230', 281)
('11364', 280)
('11795', 280)
('11228', 279)
('11233', 274)
('11729', 271)
('11378', 267)
('11368', 262)
('11375', 261)
('11419', 260)
('11365', 250)
('11373', 250)
('11432', 250)
('11768', 248)
('11218', 246)
('11355', 241)
('11212', 230)
('11361', 230)
('11422', 227)
('11433', 223)
('11435', 221)
('11367', 

- Most of postcodes start from 10 or 11. 
- And some of them start from 08 or 07 or 06 and they mean New Jersey or Connecticut. I used [these](http://www.geonames.org/postal-codes/postal-codes-us.html) [sites](https://en.wikipedia.org/wiki/New_York_metropolitan_area). Therefore I can infer that this New York OSM data has information about not just New York state, but New York Metropolitan area.
- There are 3 different types of postal codes. First, 5 digit code. Second, 5 digit code added by 4 digits connected by '-'. Third, 5 digit code started by 'NY '.I think that I can programmatically unify them. I'm going to use 5 digit code and zip+4 code both because zip+4 code is better at mailing([reference](https://smartystreets.com/articles/zip-4-code))
- There are 2 or 3 or 6 digit code and I think that it was input error. When cleaning the data before sending into DB, I'll ignore them.

In [3]:
#query about emails.
query = "select tags.value \
        from (select * from nodes_tags \
                union all \
                select * from ways_tags) tags \
        where tags.key = 'email' \
        order by tags.value;"
c.execute(query)
rows = c.fetchall()
for row in rows:
    print(row)

('bathenhaven@yahoo.com',)
('info@jadisnyc.com',)
('info@marrakechhotelnyc.com',)
('james@gsalehouse.com',)
('madison@cluizel.us',)
('support@laserengravenj.com',)


- There are just 6 emails in this sample and there isn't any wrong in them.

In [4]:
#query about facebooks.
query = "select tags.value \
        from (select * from nodes_tags \
                union all \
                select * from ways_tags) tags \
        where tags.key = 'facebook' \
        order by tags.value;"
c.execute(query)
rows = c.fetchall()
for row in rows:
    print(row)

('https://www.facebook.com/pages/MacDougal-St-Ale-House-Pla-Street/116550071739296',)


- There are just 1 facebook page in this sample and there isn't any wrong in it.

In [6]:
#query about websites.
query = "select distinct tags.value \
        from (select * from nodes_tags \
                union all \
                select * from ways_tags) tags \
        where tags.key = 'website' \
        order by tags.value;"
c.execute(query)
rows = c.fetchall()
for row in rows:
    print(row)

('baileyarboretum.org',)
('bigtreebottles.com',)
('bk.com',)
('brookwoodatbayshore.com',)
('hsbc.com',)
('http://527madisonave.com/',)
('http://825thirdave.com/',)
('http://auttharosthainewyork.com',)
('http://bathenhaven.com/',)
('http://brooklynsandbox.com',)
('http://brooklynsocialbar.com/',)
('http://celticcorner.us/',)
('http://charno4.com/',)
('http://citibikenyc.com',)
('http://davoodzadehrugs.com',)
('http://doriangraynyc.com',)
('http://drambar.com/',)
('http://ezcesspoollongisland.com',)
('http://fkh.clarkschools.org/',)
('http://gomusicworks.com',)
('http://greenwichlocksmiths.com',)
('http://gsalehouse.com/',)
('http://jadisnyc.com/',)
('http://jerichoanimalhospital.com',)
('http://keyportlibrary.org/',)
('http://kfsmortgage.com',)
('http://landmarklivingston.com/',)
('http://lennysnyc.com/stores#lenny-s-55th-street',)
('http://lfstores.com/',)
('http://livinglotsnyc.org/lot/58276/',)
('http://location.ctownsupermarkets.com/ny/jamaica/41-003/',)
('http://lucasstamford.com/'

- There are three different forms of website addresses. First, URL started from 'http' or 'https' scheme. Second, domain name started from 'www'. Third, domain name without 'www'. I think that it is better to make all website addresses which don't start with 'http' or 'https' start with 'http://' for the consistency. And the other reason is that even if there are some sites which don't use 'www' subdomain, there isn't any websites which can be accessed without using http protocol. Referenced [these](https://en.wikipedia.org/wiki/URL) [sites](https://stackoverflow.com/questions/8951423/html-links-without-http-protocol).

In [7]:
#query about phones.
query = "select distinct tags.value \
        from (select * from nodes_tags \
                union all \
                select * from ways_tags) tags \
        where tags.key = 'phone' \
        order by tags.value;"
c.execute(query)
rows = c.fetchall()
for row in rows:
    print(row)

('(201) 444-8111',)
('(201) 612-0756',)
('(212) 260-8015',)
('(212) 340-0863',)
('(212) 675-0550',)
('(212) 982-0362',)
('(212)228-5483',)
('(347) 799-2640',)
('(516) 431-5431',)
('(631) 499-8580',)
('(718) 229-7600',)
('(718) 230-3881',)
('(718) 321-6893',)
('(718) 369-0033',)
('(718) 381-5136',)
('(718) 488-7005',)
('(718) 623-9065',)
('(718) 677-5811',)
('(718) 821-8085',)
('(718) 855-9600',)
('(732) 264-7983',)
('(855) 417-1284',)
('(973) 256-7742',)
('(973) 310-3650',)
('(973) 564-9092',)
('+1 (201) 398-0944',)
('+1 (516) 801-6544',)
('+1 (718) 250-0200',)
('+1 (718) 387-9617',)
('+1 203 863 3000',)
('+1 205 9007007',)
('+1 212 2222954',)
('+1 212 242 6200',)
('+1 212 254 0006',)
('+1 212 353-3780',)
('+1 212 502 4609',)
('+1 212 663 1580',)
('+1 212 663 6004',)
('+1 212 665 9800',)
('+1 212 675-0342',)
('+1 212 690 4000',)
('+1 212 749 1111',)
('+1 212 799-9010',)
('+1 212 874 4000',)
('+1 212 9660290',)
('+1 212 989 8737',)
('+1 212-254-1675',)
('+1 516 783 86 60',)
('+1 646 415

- There are lots of different types of phone numbers in the result. I'm going to make them have ['(NPA) NXX-XXXX' format](https://en.wikipedia.org/wiki/National_conventions_for_writing_telephone_numbers), where NPA is the numbering plan area code and NXX-XXXX is the subscriber number.

In [9]:
#query about opening-hours.
query = "select distinct tags.value \
        from (select * from nodes_tags \
                union all \
                select * from ways_tags) tags \
        where tags.key = 'opening_hours' \
        order by tags.value;"
c.execute(query)
rows = c.fetchall()
for row in rows:
    print(row)

('11:00 AM to Close',)
('24/7',)
('5pm - 9pm (weekdays) , 2pm - 8pm ( weekend)',)
('772',)
('Mo-Fr 05:00-21:00, Su 06:30-21:00, Sa 06:30-21:00, PH 07:00-19:00',)
('Mo-Fr 05:30-21:30; Sa-Su 07:00-21:30',)
('Mo-Fr 05:30-22:00; Sa-Su 06:00-22:00',)
('Mo-Fr 07:00-19:00; Sa-Su 08:00-16:00',)
('Mo-Fr 07:00-20:00; Sa-Su 08:00-20:00',)
('Mo-Fr 07:00-21:00; Sa 08:00-20:00; Su 09:00-19:00',)
('Mo-Fr 07:30-18:00; Sa,Su 08:00-16:00',)
('Mo-Fr 07:30-19:00; Sa 07:30-16:00; Su 11:00-15:00',)
('Mo-Fr 08:00-18:00',)
('Mo-Fr 08:30-17:00; Sa 10:00-14:00',)
('Mo-Fr 08:30-18:00; Sa 09:00-15:00',)
('Mo-Fr 09:00-17:00',)
('Mo-Fr 09:00-20:00; Sa-Su 10:00-18:00',)
('Mo-Fr 10:00-19:30; Sa 10:00-18:00; Su 10:00-17:00',)
('Mo-Fr 10:00-20:00; Sa 10:00-18:00; Su 11:00-17:00',)
('Mo-Fr 11:00-22:00; Sa-Su 12:00-22:00',)
('Mo-Fr 12:00-16:30, 17:00-23:00; Sa-Su 12:00-23:30',)
('Mo-Fr 13:00-01:00; Sa-Su 12:00-01:00',)
('Mo-Fr 14:00-04:00',)
('Mo-Sa 07:00-19:00',)
('Mo-Sa 07:00-22:00',)
('Mo-Sa 08:00-04:00; Su 12:00-04:0

- I'll skip cleaning it. The result is too complicated for me.

In [15]:
#query about faxs.
query = "select distinct tags.value \
        from (select * from nodes_tags \
                union all \
                select * from ways_tags) tags \
        where tags.key = 'fax' \
        order by tags.value;"
c.execute(query)
rows = c.fetchall()
for row in rows:
    print(row)

('+1 212 242 1278',)
('+1 212 678 6842',)
('+1 732 5432409',)
('+1 973 761 9193',)
('+1-212-268-0847',)
('718-268-7382',)
('718-361-2734',)


In [16]:
#query about Faxs.
query = "select distinct tags.value \
        from (select * from nodes_tags \
                union all \
                select * from ways_tags) tags \
        where tags.key = 'Fax' \
        order by tags.value;"
c.execute(query)
rows = c.fetchall()
for row in rows:
    print(row)

('732-548-8887',)


- 'Fax' and 'fax' are same but divided by capital or small letter and the results are like phone number's. I'm going to alter the key 'Fax' to 'fax' and make the numbers have the same format as phone number's.

In [19]:
#query about maxspeed.
query = "select distinct tags.value \
        from (select * from nodes_tags \
                union all \
                select * from ways_tags) tags \
        where tags.key = 'maxspeed' \
        order by tags.value;"
c.execute(query)
rows = c.fetchall()
for row in rows:
    print(row)

('10',)
('10 mph',)
('110 mph',)
('125 mph',)
('15',)
('15 mph',)
('160',)
('177',)
('20 mph',)
('201',)
('217',)
('25 mph',)
('30',)
('30 mph',)
('35',)
('35 mph',)
('40 mph',)
('45 mph',)
('48',)
('50',)
('50 mph',)
('55 mph',)
('60',)
('60 mph',)
('64',)
('65 mph',)
('70 mph',)
('72',)
('75 mph',)
('80',)
('90 mph',)
('Amtrak ETT 2010-01-18',)
('Conrail ETT 2013-07-01',)
('Conrail ETT 2013-10-17',)
('signals',)


- Some of them are just number and others are number added by 'mph'. I'm going to remove unrelated datas and add 'mph' to all numbers.

In [20]:
#query about street names.
query = "select distinct tags.value \
        from (select * from nodes_tags \
                union all \
                select * from ways_tags) tags \
        where tags.key = 'street' \
        order by tags.value;"
c.execute(query)
rows = c.fetchall()
for row in rows:
    print(row)

('100th Avenue',)
('100th Drive',)
('100th Street',)
('101st Avenue',)
('101st Street',)
('102nd Avenue',)
('102nd Road',)
('102nd Street',)
('103rd Avenue',)
('103rd Road',)
('103rd Street',)
('104th Avenue',)
('104th Street',)
('105th Avenue',)
('105th Street',)
('106th Avenue',)
('106th Street',)
('107th Avenue',)
('107th Street',)
('108th Avenue',)
('108th Drive',)
('108th Street',)
('109th Avenue',)
('109th Drive',)
('109th Road',)
('109th Street',)
('10th Avenue',)
('10th Street',)
('110th Avenue',)
('110th Road',)
('110th Street',)
('111th Avenue',)
('111th Road',)
('111th Street',)
('112th Avenue',)
('112th Place',)
('112th Road',)
('112th Street',)
('113th Avenue',)
('113th Drive',)
('113th Road',)
('113th Street',)
('114th Avenue',)
('114th Drive',)
('114th Place',)
('114th Road',)
('114th Street',)
('114th Terrace',)
('115th Avenue',)
('115th Court',)
('115th Drive',)
('115th Road',)
('115th Street',)
('116th Avenue',)
('116th Drive',)
('116th Road',)
('116th Street',)
('117

('Bunnell Court',)
('Buoy Lane',)
('Burbank Avenue',)
('Burbank Court',)
('Burchell Avenue',)
('Burchell Road',)
('Burden Avenue',)
('Burden Crescent',)
('Burdette Place',)
('Burgher Avenue',)
('Burke Avenue',)
('Burling Lane S',)
('Burling Street',)
('Burlington Avenue',)
('Burnage Lane',)
('Burnett Avenue',)
('Burnett Place',)
('Burnett Street',)
('Burns Street',)
('Burnside Avenue',)
('Burr Avenue',)
('Burr Road',)
('Burrs Lane',)
('Burt Avenue',)
('Burton Lane',)
('Burton Street',)
('Bush Avenue',)
('Bush Street',)
('Bushwick Avenue',)
('Bushwick Court',)
('Bushwick Place',)
('Bussing Avenue',)
('Butler Avenue',)
('Butler Place',)
('Butler Street',)
('Buttercup Lane',)
('Butterfield Court',)
('Butterfield Drive',)
('Butterfly Drive',)
('Buttonwood Drive',)
('Buttrick Avenue',)
('Byrd Street',)
('Byrne Avenue',)
('Byron Avenue',)
('Byway Drive',)
('COBB ROAD',)
('COLLEGE AVENUE',)
('Cabota Avenue',)
('Cabrini Boulevard',)
('Cabriolet Lane',)
('Cactus Lane',)
('Cadman Avenue',)
('Cae

('Lily Pond Avenue',)
('Lincoln Avenue',)
('Lincoln Boulevard',)
('Lincoln Drive',)
('Lincoln Place',)
('Lincoln Road',)
('Lincoln Square',)
('Lincoln Street',)
('Lincoln Walk',)
('Linda Court',)
('Linda Lane',)
('Lindbergh Avenue',)
('Lindbergh Cir',)
('Lindbergh Street',)
('Linden Boulevard',)
('Linden Farms Road',)
('Linden Place',)
('Linden Street',)
('Lindenwood Road',)
('Lindron Avenue',)
('Linneaus Place',)
('Linton Avenue',)
('Linton Place',)
('Linwood Avenue',)
('Linwood Street',)
('Lion Lane',)
('Lion Street',)
('Lipsett Avenue',)
('Lipson Court',)
('Lisa Drive',)
('Lisa Lane',)
('Lisk Avenue',)
('Lispenard Street',)
('Liss Street',)
('Lister Cir',)
('Litchfield Avenue',)
('Little Clove Road',)
('Little East Neck Road',)
('Little East Neck Road N',)
('Little East Neck Road S',)
('Little Neck Parkway',)
('Little Neck Road',)
('Little Plains Court',)
('Little Plains Road',)
('Littlefield Avenue',)
('Livermore Avenue',)
('Liverpool Street',)
('Livingston Avenue',)
('Livingston C

('Webb Avenue',)
('Webro Road',)
('Webster Avenue',)
('Weed Avenue',)
('Weehawken Street',)
('Weeks Avenue',)
('Weeks Lane',)
('Weeks Road',)
('Weeping Cherry Lane',)
('Weiner Street',)
('Weirfield Street',)
('Weiss Place',)
('Welder Court',)
('Weldon Lane',)
('Weldon Street',)
('Weller Avenue',)
('Weller Lane',)
('Welling Court',)
('Wellington Court',)
('Wellington Road South',)
('Wellman Avenue',)
('Wells Road',)
('Wellwood Avenue',)
('Wendorf Court',)
('Wendover Drive',)
('Wendy Lane',)
('Wenlock Street',)
('Wenmore Road',)
('Wentworth Avenue',)
('Weser Avenue',)
('Wesleyan Road',)
('West 101st Street',)
('West 103rd Street',)
('West 104th Street',)
('West 105th Street',)
('West 106th Street',)
('West 107th Street',)
('West 108th Street',)
('West 109th Street',)
('West 10th Road',)
('West 10th Street',)
('West 111th Street',)
('West 112th Street',)
('West 113th Street',)
('West 114th Street',)
('West 115th Street',)
('West 116th Street',)
('West 117th Street',)
('West 118th Street',

- There are really lots of formats of street names. There's no shortened version of street that I'm going to leave them as it is.

### - Finally, I decided to solve the problems in maxspeed, website, postcode, phone, fax, and Fax fields.

In [1]:
#In this time, I'll designate k as 300. I'm going to check the effects of cleaning with this data.
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import xml.etree.cElementTree as ET  # Use cElementTree or lxml if too slow

OSM_FILE = "C:\\projects\\New_York_OSM\\New_York.osm"  # Replace this with your osm file
SAMPLE_FILE = "C:\\projects\\New_York_OSM\\New_York_small_sample.osm"

k = 300 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    print(root.tag, root.attrib)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n'.encode('utf-8'))
    output.write('<osm>\n  '.encode('utf-8'))

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>'.encode('utf-8'))

osm {'version': '0.6', 'generator': 'osmconvert 0.8.5', 'timestamp': '2017-09-03T15:00:02Z'}


In [9]:
#I got this schema from Data Wrangling course.
schema_contents = {
    'node': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'lat': {'required': True, 'type': 'float', 'coerce': float},
            'lon': {'required': True, 'type': 'float', 'coerce': float},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'node_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    },
    'way': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'way_nodes': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'node_id': {'required': True, 'type': 'integer', 'coerce': int},
                'position': {'required': True, 'type': 'integer', 'coerce': int}
            }
        }
    },
    'way_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    }
}

In [44]:
#In this time, I'll use cerberus and schema module to validate during writing csv file, and clean the certain fields' data during shaping.
#Use small sample and make cleaned sample csv files.
import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET

import cerberus

import schema

OSM_PATH = "C:\\projects\\New_York_OSM\\New_York_small_sample.osm"

NODES_PATH = "C:\\projects\\New_York_OSM\\small_samples\\small_sample_nodes.csv"
NODE_TAGS_PATH = "C:\\projects\\New_York_OSM\\small_samples\\small_sample_nodes_tags.csv"
WAYS_PATH = "C:\\projects\\New_York_OSM\\small_samples\\small_sample_ways.csv"
WAY_NODES_PATH = "C:\\projects\\New_York_OSM\\small_samples\\small_sample_ways_nodes.csv"
WAY_TAGS_PATH = "C:\\projects\\New_York_OSM\\small_samples\\small_sample_ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema_contents

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""
    pos = 0
    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    # YOUR CODE HERE
    if element.tag == 'node':

        for key, value in element.attrib.items():
            if key in node_attr_fields:
                if key == 'id':
                    nodeId = value
                node_attribs[key] = value
            
        for sub in element:
            key, value = sub.attrib['k'], sub.attrib['v']
            
            if not problem_chars.search(key):
                                        
                if LOWER_COLON.match(key):
                    type_value, key = key.split(":", 1)
                    
                    if key == 'maxspeed':
                        #I'm going to accept the values if the first two digits are numbers.
                        #But if there isn't 'mph' in the value, I'll add it.

                        if 'mph' not in value:
                            value = value + ' mph'
                        if re.match(r'^\d+ mph$', value) == False:
                            continue

                    elif key == 'website':
                        #If the value doesn't start with 'http', I'll add 'http://'.
                        if value[:4] != 'http':
                            value = 'http://' + value

                    elif key == 'postcode':
                        #If the first two digits are not numbers like 'NY', I'm going to remove them and extra spaces.
                        #And then, if the value isn't 5 digit number nor the 10 figures of zip+4 code I'll remove them.
                        if value[:2].isdigit() == False:
                            value = value[2:].strip()
                        if (len(value) == 5 and value.isdigit()) == False and \
                            (len(value) == 10 and value[:5].isdigit() and value[-4:].isdigit() and value[5] == '-') == False:
                                continue

                    elif key in ['phone', 'fax', 'Fax']:
                        #I'm going to change the key name 'Fax' to 'fax' for consistency.
                        if key == 'Fax':
                            key = 'fax'
                        #I won't accept if there is any alphabet in the value.
                        if re.search(r'[a-z|A-Z]', value):
                            continue
                        #If the first two figures are '+1' or '01' or '1 ' or '1-', I'm going to remove the part.
                        if value[:2] in ['+1', '01', '1 ', '1-']:
                            value = value[2:]
                        #Temporarily leave only numbers.
                        for ch in [' ', '-', '.', '(', ')']:
                            if ch in value:
                                value = value.replace(ch, '')
                        #If the left part of value isn't 10 digits, I'll remove them.
                        if (len(value) == 10 and value.isdigit()) == False:
                            continue
                        #Change the value to follow the phone number format.
                        value = '({}) {}-{}'.format(value[:3], value[3:6], value[6:])
                        
                    tags.append({"id" : nodeId, "key" : key, "value" : value, "type" : type_value})
                    
                
                else:
                    if key == 'maxspeed':
                        #I'm going to accept the values if the first two digits are numbers.
                        #But if there isn't 'mph' in the value, I'll add it.

                        if 'mph' not in value:
                            value = value + ' mph'
                        if re.match(r'^\d+ mph$', value) == False:
                            continue

                    elif key == 'website':
                        #If the value doesn't start with 'http', I'll add 'http://'.
                        if value[:4] != 'http':
                            value = 'http://' + value

                    elif key == 'postcode':
                        #If the first two digits are not numbers like 'NY', I'm going to remove them and extra spaces.
                        #And then, if the value isn't 5 digit number nor the 10 figures of zip+4 code I'll remove them.
                        if value[:2].isdigit() == False:
                            value = value[2:].strip()
                        if (len(value) == 5 and value.isdigit()) == False and \
                            (len(value) == 10 and value[:5].isdigit() and value[-4:].isdigit() and value[5] == '-') == False:
                                continue

                    elif key in ['phone', 'fax', 'Fax']:
                        #I'm going to change the key name 'Fax' to 'fax' for consistency.
                        if key == 'Fax':
                            key = 'fax'
                        #I won't accept if there is any alphabet in the value.
                        if re.search(r'[a-z|A-Z]', value):
                            continue
                        #If the first two figures are '+1' or '01' or '1 ' or '1-', I'm going to remove the part.
                        if value[:2] in ['+1', '01', '1 ', '1-']:
                            value = value[2:]
                        #Temporarily leave only numbers.
                        for ch in [' ', '-', '.', '(', ')']:
                            if ch in value:
                                value = value.replace(ch, '')
                        #If the left part of value isn't 10 digits, I'll remove them.
                        if (len(value) == 10 and value.isdigit()) == False:
                            continue
                        #Change the value to follow the phone number format.
                        value = '({}) {}-{}'.format(value[:3], value[3:6], value[6:])
                        
                    tags.append({"id" : nodeId, "key" : key, "value" : value, "type" : default_tag_type})
                      
        return {'node': node_attribs, 'node_tags': tags}
    
    elif element.tag == 'way':

        for key, value in element.attrib.items():
            if key in WAY_FIELDS:
                if key == 'id':
                    wayId = value
                way_attribs[key] = value
                
        for sub in element:
            if sub.tag == 'tag':
                key, value = sub.attrib['k'], sub.attrib['v']
                if not problem_chars.search(key):

                    if LOWER_COLON.match(key):
                        type_value, key = key.split(":", 1)
                        
                        if key == 'maxspeed':
                            #I'm going to accept the values if the first two digits are numbers.
                            #But if there isn't 'mph' in the value, I'll add it.

                            if 'mph' not in value:
                                value = value + ' mph'
                            if re.match(r'^\d+ mph$', value) == False:
                                continue

                        elif key == 'website':
                            #If the value doesn't start with 'http', I'll add 'http://'.
                            if value[:4] != 'http':
                                value = 'http://' + value

                        elif key == 'postcode':
                            #If the first two digits are not numbers like 'NY', I'm going to remove them and extra spaces.
                            #And then, if the value isn't 5 digit number nor the 10 figures of zip+4 code I'll remove them.
                            if value[:2].isdigit() == False:
                                value = value[2:].strip()
                            if (len(value) == 5 and value.isdigit()) == False and \
                                (len(value) == 10 and value[:5].isdigit() and value[-4:].isdigit() and value[5] == '-') == False:
                                    continue

                        elif key in ['phone', 'fax', 'Fax']:
                            #I'm going to change the key name 'Fax' to 'fax' for consistency.
                            if key == 'Fax':
                                key = 'fax'
                            #I won't accept if there is any alphabet in the value.
                            if re.search(r'[a-z|A-Z]', value):
                                continue
                            #If the first two figures are '+1' or '01' or '1 ' or '1-', I'm going to remove the part.
                            if value[:2] in ['+1', '01', '1 ', '1-']:
                                value = value[2:]
                            #Temporarily leave only numbers.
                            for ch in [' ', '-', '.', '(', ')']:
                                if ch in value:
                                    value = value.replace(ch, '')
                            #If the left part of value isn't 10 digits, I'll remove them.
                            if (len(value) == 10 and value.isdigit()) == False:
                                continue
                            #Change the value to follow the phone number format.
                            value = '({}) {}-{}'.format(value[:3], value[3:6], value[6:])
                        
                        tags.append({"id" : wayId, "key" : key, "value" : value, "type" : type_value})
                                              
                    else:
                        if key == 'maxspeed':
                            #I'm going to accept the values if the first two digits are numbers.
                            #But if there isn't 'mph' in the value, I'll add it.

                            if 'mph' not in value:
                                value = value + ' mph'
                            if re.match(r'^\d+ mph$', value) == False:
                                continue

                        elif key == 'website':
                            #If the value doesn't start with 'http', I'll add 'http://'.
                            if value[:4] != 'http':
                                value = 'http://' + value

                        elif key == 'postcode':
                            #If the first two digits are not numbers like 'NY', I'm going to remove them and extra spaces.
                            #And then, if the value isn't 5 digit number nor the 10 figures of zip+4 code I'll remove them.
                            if value[:2].isdigit() == False:
                                value = value[2:].strip()
                            if (len(value) == 5 and value.isdigit()) == False and \
                                (len(value) == 10 and value[:5].isdigit() and value[-4:].isdigit() and value[5] == '-') == False:
                                    continue

                        elif key in ['phone', 'fax', 'Fax']:
                            #I'm going to change the key name 'Fax' to 'fax' for consistency.
                            if key == 'Fax':
                                key = 'fax'
                            #I won't accept if there is any alphabet in the value.
                            if re.search(r'[a-z|A-Z]', value):
                                continue
                            #If the first two figures are '+1' or '01' or '1 ' or '1-', I'm going to remove the part.
                            if value[:2] in ['+1', '01', '1 ', '1-']:
                                value = value[2:]
                            #Temporarily leave only numbers.
                            for ch in [' ', '-', '.', '(', ')']:
                                if ch in value:
                                    value = value.replace(ch, '')
                            #If the left part of value isn't 10 digits, I'll remove them.
                            if (len(value) == 10 and value.isdigit()) == False:
                                continue
                            #Change the value to follow the phone number format.
                            value = '({}) {}-{}'.format(value[:3], value[3:6], value[6:])
                            
                        tags.append({"id" : wayId, "key" : key, "value" : value, "type" : default_tag_type})
                      
            else:
                ref = sub.attrib['ref']
                way_nodes.append({"id" : wayId, "node_id" : ref, "position" : pos})
                pos += 1
                
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}


# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))

'''
#This part of code is for Python2.
class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.items()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)
'''

# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with open(NODES_PATH, 'w', encoding='UTF-8', newline='') as nodes_file, \
         open(NODE_TAGS_PATH, 'w', encoding='UTF-8', newline='') as nodes_tags_file, \
         open(WAYS_PATH, 'w', encoding='UTF-8', newline='') as ways_file, \
         open(WAY_NODES_PATH, 'w', encoding='UTF-8', newline='') as way_nodes_file, \
         open(WAY_TAGS_PATH, 'w', encoding='UTF-8', newline='') as way_tags_file:

        nodes_writer = csv.DictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = csv.DictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = csv.DictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = csv.DictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = csv.DictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSM_PATH, validate=True)

'\n#This part of code is for Python2.\nclass UnicodeDictWriter(csv.DictWriter, object):\n    """Extend csv.DictWriter to handle Unicode input"""\n\n    def writerow(self, row):\n        super(UnicodeDictWriter, self).writerow({\n            k: (v.encode(\'utf-8\') if isinstance(v, unicode) else v) for k, v in row.items()\n        })\n\n    def writerows(self, rows):\n        for row in rows:\n            self.writerow(row)\n'

### - I'm going to make the uncleaned small sample data to know the problem of my cleaning part of code.

In [10]:
#In this time, I'll use cerberus and schema module to validate during writing csv file, and clean the certain fields' data during shaping.
#Use small sample and make csv files uncleaned.
import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET

import cerberus

import schema

OSM_PATH = "C:\\projects\\New_York_OSM\\New_York_small_sample.osm"

NODES_PATH = "C:\\projects\\New_York_OSM\\small_samples\\uncleaned_small_sample_nodes.csv"
NODE_TAGS_PATH = "C:\\projects\\New_York_OSM\\small_samples\\uncleaned_small_sample_nodes_tags.csv"
WAYS_PATH = "C:\\projects\\New_York_OSM\\small_samples\\uncleaned_small_sample_ways.csv"
WAY_NODES_PATH = "C:\\projects\\New_York_OSM\\small_samples\\uncleaned_small_sample_ways_nodes.csv"
WAY_TAGS_PATH = "C:\\projects\\New_York_OSM\\small_samples\\uncleaned_small_sample_ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema_contents

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    # YOUR CODE HERE
    if element.tag == 'node':
        for key, value in element.attrib.items():
            if key in node_attr_fields:
                if key == 'id':
                    nodeId = value
                node_attribs[key] = value
            
        for sub in element:
            key, value = sub.attrib['k'], sub.attrib['v']
            
            if not problem_chars.search(key):
                                 
                if LOWER_COLON.match(key):
                    left, right = key.split(":", 1)
                    tags.append({"id" : nodeId, "key" : right, "value" : value, "type" : left})
                    #print('node tag', left, right, value)
                else:
                    tags.append({"id" : nodeId, "key" : key, "value" : value, "type" : default_tag_type})
                         
        return {'node': node_attribs, 'node_tags': tags}
    
    elif element.tag == 'way':
        for key, value in element.attrib.items():
            if key in WAY_FIELDS:
                if key == 'id':
                    wayId = value
                way_attribs[key] = value
                
        pos = 0
        for sub in element:
            if sub.tag == 'tag':
                key, value = sub.attrib['k'], sub.attrib['v']
                if not problem_chars.search(key):
                   
                    if LOWER_COLON.match(key):
                        left, right = key.split(":", 1)
                        tags.append({"id" : wayId, "key" : right, "value" : value, "type" : left})
                    
                    else:
                        tags.append({"id" : wayId, "key" : key, "value" : value, "type" : default_tag_type})
                        
            else:
                ref = sub.attrib['ref']
                way_nodes.append({"id" : wayId, "node_id" : ref, "position" : pos})
                pos += 1
                
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}


# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))

'''
#This part of code is for Python2.
class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.items()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)
'''

# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with open(NODES_PATH, 'w', encoding='UTF-8', newline='') as nodes_file, \
         open(NODE_TAGS_PATH, 'w', encoding='UTF-8', newline='') as nodes_tags_file, \
         open(WAYS_PATH, 'w', encoding='UTF-8', newline='') as ways_file, \
         open(WAY_NODES_PATH, 'w', encoding='UTF-8', newline='') as way_nodes_file, \
         open(WAY_TAGS_PATH, 'w', encoding='UTF-8', newline='') as way_tags_file:

        nodes_writer = csv.DictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = csv.DictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = csv.DictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = csv.DictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = csv.DictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSM_PATH, validate=True)

'\n#This part of code is for Python2.\nclass UnicodeDictWriter(csv.DictWriter, object):\n    """Extend csv.DictWriter to handle Unicode input"""\n\n    def writerow(self, row):\n        super(UnicodeDictWriter, self).writerow({\n            k: (v.encode(\'utf-8\') if isinstance(v, unicode) else v) for k, v in row.items()\n        })\n\n    def writerows(self, rows):\n        for row in rows:\n            self.writerow(row)\n'